In [2]:
from typing import Annotated
from dotenv import load_dotenv
import os

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI  # Correct Gemini wrapper

# Load API key from .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# Initialize Gemini LLM with API key
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=api_key
)

# Define state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Chatbot node
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

# Build graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

# Single-turn example
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
response = graph.invoke({"messages": [message]})
print("Bot:", response["messages"][-1].content)

# Multi-turn interactive loop
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {"messages": [{"role": "user", "content": in_message}]}
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)


Bot: Neil Armstrong
Bot: Neil Armstrong's first words upon stepping onto the Moon were: "That's one small step for [a] man, one giant leap for mankind."
Bot: Neil Armstrong's "one small step" quote is considered inspirational for a number of reasons:

*   **Perspective:** It puts a single, personal achievement into the context of something much larger. It's not just about him or the astronauts; it's about the progress of humanity.

*   **Hope and Possibility:** The phrase "giant leap" suggests overcoming immense challenges and achieving the seemingly impossible. This inspires people to believe in their own ability to accomplish great things, even when faced with obstacles.

*   **Unity:** By framing the event as a leap for "mankind," it emphasizes shared human aspirations and the potential for collective achievement. It transcends national boundaries and ideological differences.

*   **Simplicity and Elegance:** The language is simple and direct, yet profound. It's easy to remember and

/home/mahad/Desktop/Codes/venv/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:1805: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


Bot: 


/home/mahad/Desktop/Codes/venv/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:1805: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


KeyboardInterrupt: 